In [ ]:
# Boilerplate that all notebooks reuse:
from analysis_common import *

%matplotlib inline

# Kernel analysis

In [ ]:
df = read_ods("./results.ods", "kmeans-kernel")
expand_modes(df)

sns.barplot(x='MODE', y='TIMING',
            data=df[(df.POINTS_PER_FRAGMENT == 200000)],
            capsize=0.1,
            order=['AD', 'AD-NVM', 'MM-DRAM', 'MM-NVM'],
            palette=palette)
plt.title("POINTS: 200k")
plt.show()


sns.barplot(x='MODE', y='TIMING',
            data=df[(df.POINTS_PER_FRAGMENT == 4000)],
            capsize=0.1,
            order=['AD', 'AD-NVM', 'MM-DRAM', 'MM-NVM'],
            palette=palette)
plt.title("POINTS: 4k")
plt.show()

n_n_points = 100000
normalized_value = n_n_points / df[(df.MODE == "AD") & (df.POINTS_PER_FRAGMENT == n_n_points)]["TIMING"].mean()
df["NORMALIZED"] = normalized_value * df["TIMING"] / df["POINTS_PER_FRAGMENT"]


sns.catplot(x='POINTS_PER_FRAGMENT', y='NORMALIZED', hue='MODE',
            capsize=0.1, kind='bar', data=df, palette=palette)
plt.show()

In [ ]:
kernel_mean_times = df.groupby(["POINTS_PER_FRAGMENT", "MODE"]).mean()
kernel_mean_times

# _k_-means results analysis

In [ ]:
df = read_ods("./results.ods", "kmeans-app")
expand_modes(df)

In [ ]:
for ppf in [200000, 4000]:
    df.loc[(df.POINTS_PER_FRAGMENT == ppf) & (df.MODE == "AD"), "ATOM_KERNEL"] =  \
        kernel_mean_times.loc[(ppf, "AD"), "TIMING"]
    df.loc[(df.POINTS_PER_FRAGMENT == ppf) & (df.MODE == "AD-NVM"), "ATOM_KERNEL"] =  \
        kernel_mean_times.loc[(ppf, "AD-NVM"), "TIMING"]
    df.loc[(df.POINTS_PER_FRAGMENT == ppf) & (df.MODE == "DAOS"), "ATOM_KERNEL"] =  \
        kernel_mean_times.loc[(ppf, "AD"), "TIMING"]
    df.loc[(df.POINTS_PER_FRAGMENT == ppf) & (df.MODE == "DC-NONACT"), "ATOM_KERNEL"] = \
        kernel_mean_times.loc[(ppf, "AD"), "TIMING"]

df["KERNEL_TIME"] = df["NUMBER_OF_FRAGMENTS"] * df["KMEANS_ITERATIONS"] * df["ATOM_KERNEL"]

# Sanity check
null_values = df[df.isnull().values]
if len(null_values) > 0:
    null_values # show that

In [ ]:
small = (
    ((df.POINTS_PER_FRAGMENT == 200000) & (df.NUMBER_OF_FRAGMENTS == 16)) |
    ((df.POINTS_PER_FRAGMENT == 4000) & (df.NUMBER_OF_FRAGMENTS == 800))
)

big = (
    ((df.POINTS_PER_FRAGMENT == 200000) & (df.NUMBER_OF_FRAGMENTS == 64)) |
    ((df.POINTS_PER_FRAGMENT == 4000) & (df.NUMBER_OF_FRAGMENTS == 3200))
)

sns.barplot(x='MODE', y="TIMING",
            data=df[small],
            capsize=0.1,
            hue=df.POINTS_PER_FRAGMENT)

bottom = sns.barplot(x='MODE', y="KERNEL_TIME",
            data=df[small],
            capsize=0,
            color="red",
            alpha=0.6,
            hue=df.POINTS_PER_FRAGMENT)

special_cols = len(bottom.patches) // 2
for i, bar in enumerate(bottom.patches[-special_cols:]):
    current_width = bar.get_width()
    diff = 0.05
    # we change the bar width
    bar.set_width(current_width + diff)

    # we recenter the bar
    if i < (special_cols // 2):
        bar.set_x(bar.get_x() - diff)
plt.show()

sns.barplot(x='MODE', y="TIMING",
            data=df[big],
            capsize=0.1,
            hue=df.POINTS_PER_FRAGMENT)

bottom = sns.barplot(x='MODE', y="KERNEL_TIME",
            data=df[big],
            capsize=0,
            color="blue",
            alpha=0.6,
            hue=df.POINTS_PER_FRAGMENT)

special_cols = len(bottom.patches) // 2
for i, bar in enumerate(bottom.patches[-special_cols:]):
    current_width = bar.get_width()
    diff = 0.05
    # we change the bar width
    bar.set_width(current_width + diff)

    # we recenter the bar
    if i < (special_cols // 2):
        bar.set_x(bar.get_x() - diff)
plt.show()